In [70]:
import sys
sys.path.append('../../../')
import torch
from matplotlib import pyplot as plt
from tifffile import imread, imwrite
from sklearn.cluster import KMeans
import numpy as np
from glob import glob
from tqdm import tqdm

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
path="/group/jug/Sheida/pancreatic beta cells/download/high_c1/"
patch_size = 64
model = torch.load("/group/jug/Sheida/HDN models/13122023/HDN Muller_best_vae.net")
model.mode_pred=True
model.eval()
model.to(device)

LadderVAE(
  (first_bottom_up): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ELU(alpha=1.0)
    (2): BottomUpDeterministicResBlock(
      (res): ResidualBlock(
        (block): Sequential(
          (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ELU(alpha=1.0)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): Dropout2d(p=0.2, inplace=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ELU(alpha=1.0)
          (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (7): Dropout2d(p=0.2, inplace=False)
        )
      )
    )
  )
  (top_down_layers): ModuleList(
    (0-3): 4 x TopDownLayer(
      (deterministic_block): Sequential(
        (0): TopDownDeterministicResBlock(
          (pre_conv): ConvTranspose2d(64, 64, kernel_size=(3, 3), stride=(2, 2), 

In [4]:
def clustering(mu):
    slice_features_mu = mu.astype(float)
    feature_flatten_mu = slice_features_mu.reshape(160, -1).T
    K_CENTRE = 8
    kmeans_mu = KMeans(
        n_clusters=K_CENTRE, init='k-means++', n_init=10,
        max_iter=1000, random_state=777
    )
    kmeans_mu.fit(feature_flatten_mu)
    labels_mu = kmeans_mu.predict(feature_flatten_mu)
    return labels_mu

In [5]:
def get_normalized_tensor(img,model,device):
    '''
    Normalizes tensor with mean and std.
    Parameters
    ----------
    img: array
        Image.
    model: Hierarchical DivNoising model
    device: GPU device.
    '''
    test_images = torch.from_numpy(img.copy()).to(device)
    data_mean = model.data_mean
    data_std = model.data_std
    test_images = (test_images-data_mean)/data_std
    return test_images

In [63]:
def make_tiled(img):
    out = np.zeros([img.shape[0]+61, img.shape[1]+60])
    # make mirror margin of 2d image
    for i in range(30):
        out[i,30:30+img.shape[1]] = img[30-i-1,:]
        out[30:30+img.shape[0],30:30+img.shape[1]] = img
        out[30+img.shape[0]+i,30:30+img.shape[1]] = img[img.shape[0]-i-1,:]
    out[img.shape[0]+60,30:30+img.shape[1]] = img[img.shape[0]-31,:]
    for i in range(30):
        out[:,i] = out[:,60-i-1]
        out[:, 30+img.shape[1]+i] = out[:, 30+img.shape[1]-i-1]
    return out
    

In [67]:
img = imread(path+"data/test/high_c1_source_661.tif")
tiled = make_tiled(img)
img_height,img_width = 700,760
img = get_normalized_tensor(tiled,model,device)
output = np.zeros((160, 700, 760))
for i in tqdm(range(0, img_height, 4)):
    for j in range(0, img_width, 4):
        img_t = img[i:i+64,j:j+64]
        image_sample = img_t.view(1,1,64,64)
        image_sample = image_sample.to(device=device, dtype=torch.float)
        with torch.no_grad():
            sample = model(image_sample)
            mus = sample['mu']
            output[:32,i:i+4,j:j+4] = sample['mu'][0][0].cpu().numpy().repeat(2,axis=1).repeat(2,axis=2)[:,30:34,30:34]
            output[32:64,i:i+4,j:j+4] = sample['mu'][1][0].cpu().numpy().repeat(4,axis=1).repeat(4,axis=2)[:,30:34,30:34]
            output[64:96,i:i+4,j:j+4] = sample['mu'][2][0].cpu().numpy().repeat(8,axis=1).repeat(8,axis=2)[:,30:34,30:34]
            output[96:128,i:i+4,j:j+4] = sample['mu'][3][0].cpu().numpy().repeat(16,axis=1).repeat(16,axis=2)[:,30:34,30:34]
            output[128:160,i:i+4,j:j+4] = sample['mu'][4][0].cpu().numpy().repeat(32,axis=1).repeat(32,axis=2)[:,30:34,30:34]
clusters = clustering(output)

In [71]:
img = imread(path+"data/test/high_c1_source_661.tif")
tiled = make_tiled(img)
img_height,img_width = 700,760
img = get_normalized_tensor(tiled,model,device)
output = np.zeros((160, 700, 760))
for i in tqdm(range(0, img_height, 4)):
    for j in range(0, img_width, 4):
        img_t = img[i:i+64,j:j+64]
        image_sample = img_t.view(1,1,64,64)
        image_sample = image_sample.to(device=device, dtype=torch.float)
        with torch.no_grad():
            sample = model(image_sample)
            mus = sample['mu']
            output[:32,i:i+4,j:j+4] = sample['mu'][0][0].cpu().numpy()[:,14:18,14:18]
            output[32:64,i:i+4,j:j+4] = sample['mu'][1][0].cpu().numpy()[:,6:10,6:10]
            output[64:96,i:i+4,j:j+4] = sample['mu'][2][0].cpu().numpy()[:,2:6,2:6]
            output[96:128,i:i+4,j:j+4] = sample['mu'][3][0].cpu().numpy()
            output[128:160,i:i+4,j:j+4] = sample['mu'][4][0].cpu().numpy().repeat(2,axis=1).repeat(2,axis=2)
clusters = clustering(output)

  0%|          | 0/175 [00:00<?, ?it/s]

100%|██████████| 175/175 [28:19<00:00,  9.71s/it]


In [68]:
imwrite(path+"x.tif", clusters.reshape(700, 760))

In [10]:
files = glob(path+"data/test/high_c1_source_31.tif")
img_height,img_width = 699, 760
for i in files:
    img = imread(i)
    img_t = get_normalized_tensor(img,model,device)
    image_sample = img_t.view(1,1,img_height,img_width)
    image_sample = image_sample.to(device=device, dtype=torch.float)
    with torch.no_grad():
        sample = model(image_sample)
        mus = sample['mu']
        output = np.zeros((160, 768, 704))

        print(output.shape)

        for idx in range(len(sample['mu'])):
            mu = sample['mu'][idx][0].cpu().numpy()
            # print(mu.shape)
            # output = clustering(idx, mu)
            # print(i)
            # imwrite(path+"label_model_21012024/k = 8/"+str(idx+1)+"/"+i[i.rfind("/"):i.rfind(".")]+".tif", output.reshape(22*(2**(4-idx)), 24*(2**(4-idx))))
            # plt.imshow(output.reshape(22*(2**(4-idx)), 24*(2**(4-idx))))
            # plt.show()

(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)
(160, 768, 704)


KeyboardInterrupt: 